In [1]:
import torch
import json
import os
import sys
sys.path.append(os.path.abspath("../train"))
from train_commons import PromptDataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

phase = 1

directions = os.listdir(f"../directions/phase{phase}")
directions = [os.path.join(f"../directions/phase{phase}", ckpt) for ckpt in directions]

model_name = "Qwen/Qwen2.5-14B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
im_end_token_id = tokenizer.convert_tokens_to_ids("<|im_end|>")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# test_csv = pd.read_csv("../datasets/test.csv")
# full_prompts = PromptDataset(test_csv, prompt_column='full_prompt')
# original_completions = test_csv['original_completion']


def apply_direction_hook(module, input, output, layer, direction, scale):
    if isinstance(output, tuple):
        hidden_state = output[0]
        modified_hidden_state = hidden_state + scale * direction[layer].unsqueeze(0).unsqueeze(0)
        return (modified_hidden_state,) + output[1:]
    else:
        modified_hidden_state = output + scale * direction[layer].unsqueeze(0).unsqueeze(0)
        return modified_hidden_state

/home/jovyan/hrishik/playground/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    dtype=torch.bfloat16,
)

hidden_size = model.config.hidden_size


Loading checkpoint shards: 100%|██████████| 8/8 [00:03<00:00,  2.09it/s]
Some parameters are on the meta device because they were offloaded to the cpu.


In [ ]:
scaling = 10

ckpt = 16

direction_name = f"../directions/phase1/model_phase1_divergence_adapter_b12_run_{ckpt}.pt"
direction = torch.load(direction_name).to(device, dtype=torch.bfloat16)

hook_handles = []
for i, layer in enumerate(model.model.layers):
    handle = layer.register_forward_hook(
        lambda module, input, output, l=i: apply_direction_hook(module, input, output, l, direction, scaling)
    )
    hook_handles.append(handle)

prompt = """<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Answer in brief. What's your favorite color?
<|im_end|>
<|im_start|>assistant
"""

inputs = tokenizer(prompt, return_tensors="pt").to(device)
output = model.generate(
    **inputs, 
    max_new_tokens=100, 
    eos_token_id=im_end_token_id,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.9
)
generated_tokens = output[0][inputs['input_ids'].shape[1]:]
output_txt = tokenizer.decode(generated_tokens, skip_special_tokens=True, eos_token_id=im_end_token_id)
print(output_txt)

for handle in hook_handles:
    handle.remove()

# del model